# Subito scraper



## Avanzato

In [1]:
from pprint import pprint
from time import sleep
from random import randint
from collections import defaultdict
import requests
from bs4 import BeautifulSoup

online = False

search_url = 'https://www.subito.it/annunci-italia/vendita/usato/'

# NOTA: Se non è indicato, è "Dal più recente"
order_opts = ['priceasc', 'pricedesc', 'relevance']

# Impostazioni di ricerca per lo scraper
search_query = {
    'search': 'sinclair+spectrum',
    'order': 'relevance',
    'max_page': None
}

if online or not 'res_cache' in globals():
    # res_cache = defaultdict(type(None))
    res_cache = defaultdict(lambda: None)

# Funzione di estrazione articoli su una singola pagina della ricerca
def scrape_page(page):
    ## Send Request
    query_str = {
        'q': search_query['search'],
        'o': page,
        'order': search_query['order']
    }

    response = requests.get(search_url, query_str)

    # Controlla che il server abbia risposto positivamente
    if not response.status_code == 200:
        raise requests.RequestException(f'Pagina {response.url} non trovata!')
 
    ## Parse page
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Controlla che non sia la pagina "Nessun risultato al momento..."
    no_results = soup.select('div[class*=ErrorLayout_container__]')
    if no_results:
        raise requests.RequestException('Non ci sono più pagine da analizzare.') 

    # Rilegge il numero di risultati totali (@todo)
    tot_ads = soup.select('p[class*=AdsCount_total-ads__]')[0].get_text()

    products = soup.select('div.items__item.item-card')
    items = []

    for product in products:
        data = {
            'name': product.select('h2[class*=ItemTitle-module_item-title__]'),
            'town': product.select('span[class*=index-module_town__]'),
            'city': product.select('span[class*=index-module_token-caption__].city'),
            'date': product.select('span[class*=index-module_date__]'),
            'price': product.select('p[class*=index-module_price__].price'),
            'shipping': product.select('span.shipping-badge'),
            'sold': product.select('span.item-sold-badge'),
        }

        for key, val in data.items():
            if val:
                data[key] = val[0].get_text()
            else:
                data[key] = None

            # data[key] = val[0].find(text=True, recursive=False) if val else None
            # data[key] = val[0].contents[0] if val else None

        data.update({
            'price': data['price'].replace('\xa0', ' ').replace('Spedizione disponibile', ''),
            'url': product.select('a.link')[0]['href'],  
        })

        items.append(data)
    
    return tot_ads, items
    

# Contatore per le pagine
curr_page = 1
# Contenitore per i dati estratti
scraped_data = []
while True:
    # Se la max_page non è impostata oppure non si è ancora raggiunta la max_page
    if not search_query['max_page'] or search_query['max_page'] >= curr_page:
        try:
            tot_ads, items = scrape_page(curr_page)
        except requests.RequestException as err:
            print(err)
            break
        
        scraped_data += items
        curr_page += 1
        sleep(randint(1, 4))
    # (Se la max_page è impostata e si è raggiunta)
    else:
        break

print(tot_ads)
pprint(scraped_data)

full_data = {
    'search': '',
    'order': '',
    # 'search_url': '',
    'tot_ads': '',
    'max_page': '',
    'timestamp': '',
    'items': '',
}

AttributeError: 'NoneType' object has no attribute 'replace'